# Proxy checker

In [2]:
with open("proxies.csv", "r") as f:
    proxies = f.read().split()
    
print(len(proxies))
print(len(set(proxies)))

with open("proxies.csv", "w") as f:
    f.write("\n".join(sorted(set(proxies))))

2516
2516


In [2]:
import requests
from lxml import html
import queue
import threading
    
NUM_THREADS = 300

def check_proxies():   
    
    tasks = queue.Queue() 
    responses = queue.Queue() 
    
    with open("proxies.csv", "r") as f:
        proxies = f.read().split()
    
    for p in proxies:
        tasks.put(p)
    
    def worker():
        while True:
            proxy = tasks.get()
            if proxy is None:
                break 
                
            try:
                base_url = "http://www.nasdaq.com/symbol/{symbol}/time-sales?time={time}&pageno={pageno}"
                url = base_url.format(symbol="AAPL", time=1, pageno=1)
                page_res = requests.get(url, timeout=6, proxies={'http': "http://{}".format(proxy)})
                tree = html.fromstring(page_res.content)
            except Exception as e:
                pass
            else:
                try:
                    select = tree.find('.//select[@id="quotes_content_left_DropDowntimerange"]')
                    if select is None or int(select.value) != 1:
                        pass
                    else:        
                        tree = html.fromstring(page_res.content)
                        table = tree.find('.//table[@id="AfterHoursPagingContents_Table"]')
                        if table is None:
                            pass
                        else:
                            responses.put(proxy)
                except Exception:
                    print(e)
                 
            tasks.task_done()
    

    threads = []
    for i in range(NUM_THREADS):
        t = threading.Thread(target=worker)
        t.start()
        threads.append(t)

    # block until all tasks are done
    tasks.join()

    # stop workers
    for i in range(NUM_THREADS):
        tasks.put(None)

    for t in threads:
        t.join()
        
    approved = []
    while True:
        try:
            proxy = responses.get(block=False)
        except queue.Empty:
            break
        else:
            approved.append(proxy)
            
    print(len(approved))
    with open("approved_proxies.csv", "w") as f:
        f.write("\n".join(approved))

check_proxies()

516
